In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
from dash_core_components import Dropdown
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "static"
shelter = AnimalShelter(username, password, 'ACC')


# class read method must support return of cursor object 
default = {}
selectedValue = {}

df = pd.DataFrame.from_records(shelter.returnAll(selectedValue,0))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' #Grazioso File
encoded_image = base64.b64encode(open(image_filename, 'rb').read()) #encodes img in Base64


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),

    #IMAGES
    html.Center(
        children = [
            html.A([            #makes this a link
                html.Img(                                    
                    src='data:image/png;base64,{}'.format(encoded_image.decode()), #sets the img                                   
                    height = 150,                                    
                    width = 150                
                ),
            ],href = 'https://www.snhu.edu' #link redirect link
            ),
            html.B(
                children = [                
                    html.H1(                    
                        children = 'SNHU CS-340 Dashboard',                                                                   
                        style = {'marginTop':50,'marginBelow':-50,'width':'2350px'}                       
                    ),
                    html.Label(
                        'Created By: David Anguiano', #Unique Identifier
                        style = {'marginTop':-20,'width':'2350px'}
                    )
                ]
            ),
             html.A([            
                html.Img(                                    
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),                                    
                    height = 150,                                    
                    width = 150                
                ),
            ],href = 'https://www.snhu.edu'
            ),
        ],
        style={'display':'flex','flex-direciton':'row'}
    ),
    html.Hr(),
        
    
    html.Div(
    id='test'
    ),
    
    #FILTER TEXT
    html.Div([    
        html.Div(
            children=[            
                html.Label('Filters'),
                #FILTER DROPDOWN        
                Dropdown(        
                    id='rescue-filters',        
                    options=[                
                        {'label':'All','value':0},                
                        {'label':'Water Rescue','value':1},                
                        {'label':'Mountain or Wilderness Rescue','value':2},                
                        {'label':'Disaster Rescue or Individual Tracking','value':3}            
                    ],            
                    value = 0   
                )        
            ], 
                style={'padding':10, 'flex':1}
            ),
        #RESET BUTTON
            #html.Div(        
              #  html.Button(id='reset',n_clicks=0,children='Reset',value = 0),       
               # style = {'marginTop':35}
    #)
        ],
    style={'display':'flex','flex-direciton':'row'}),
    
    html.Hr(),
    
    ##  DATA TABLE
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        style_data_conditional=[],
        page_action="native",
        page_current=0,
        page_size=10,
        #data=df.to_dict('records'),
        
    ),
    
    
    ## CHARTS
    html.Div([
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    ####GEOLOCATION MAP
    html.Div(
            id = 'map-id',
            className ='col s12 m6',
            children=[
                # Austin TX is at [30.75,-97.48]
                dl.Map(
                    id = 'map-update',
                    style={'width': '1175px', 'height': '500px'}, 
                    center=[df.iloc[0,13],df.iloc[0,14]], 
                    zoom=10, 
                    children=[            
                    ])
            ]
            ),
        
        ## BAR CHART
        html.Div(
            dcc.Graph(            
                id='bar-chart',
                style = {'width': '1175px', 'height': '500px'},
            )
        )
    ],
        style={'display':'flex','flex-direciton':'row'}
    ),
    
    html.Br(),
    html.Hr(),
])

#############################################
# Interaction Between Components / Controller
#############################################


##MAP UPDATES
@app.callback(
    Output('map-update','children'),
    [
        Input('rescue-filters','value'),
    ]
)

def update_map(value):
    if int(value)==0:
        data = {}
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
        markers =[dl.TileLayer(id="base-layer-id"),
        dl.Marker(
                    position=[df.iloc[0,13],df.iloc[0,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[0,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[0,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[1,13],df.iloc[1,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[1,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[1,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[2,13],df.iloc[2,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[2,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[2,9])
                      ])
                  ],
                )]
    elif int(value)==1:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["Chesapeake Bay Retriever","Labrador Retriever Mix","Newfoundland"]
            },
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
        markers =[dl.TileLayer(id="base-layer-id"),
        dl.Marker(
                    position=[df.iloc[0,13],df.iloc[0,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[0,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[0,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[1,13],df.iloc[1,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[1,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[1,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[2,13],df.iloc[2,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[2,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[2,9])
                      ])
                  ],
                )]
    elif int(value)==2:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]
            },
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
        markers =[dl.TileLayer(id="base-layer-id"),
        dl.Marker(
                    position=[df.iloc[0,13],df.iloc[0,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[0,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[0,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[1,13],df.iloc[1,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[1,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[1,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[2,13],df.iloc[2,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[2,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[2,9])
                      ])
                  ],
                )]
    elif int(value)==3:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]
            },
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20,"$lte":300}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,0)),
        markers =[dl.TileLayer(id="base-layer-id"),
        dl.Marker(
                    position=[df.iloc[0,13],df.iloc[0,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[0,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[0,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[1,13],df.iloc[1,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[1,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[1,9])
                      ])
                  ],
                ),
           dl.Marker(
                    position=[df.iloc[2,13],df.iloc[2,14]],
                    children=[                                                                      
                      dl.Tooltip(df.iloc[2,4]),                                                  
                      dl.Popup([                
                          html.H1("Animal Name"),                
                          html.P(df.iloc[2,9])
                      ])
                  ],
                )]
    return markers



## BAR CHART 
@app.callback(
    Output('bar-chart','figure'),
    [
        Input('rescue-filters','value'),
    ],   
)

def update_barchart(value):
    if int(value)==0:
        data = {}
        df = pd.DataFrame.from_records(shelter.returnAll(data,25))
        dff = df
        titleLabels =['All','Water Rescue','Mountain or Wilderness Rescue','Disaster Rescue or Individual Tracking']
        titleVal = titleLabels[value]
        barchart=px.bar(        
            data_frame=dff,        
            x='name',
            y='age_upon_outcome_in_weeks', 
            labels={
                'name':'Names',
                'age_upon_outcome_in_weeks':'Age',
                'breed':'Breed',
                'outcome_type':'Status'
            },
            hover_data = {
                'age_upon_outcome_in_weeks',
                'name',
                'breed',
                'outcome_type'
            },
            title=titleVal
    )
    elif int(value)==1:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["Chesapeake Bay Retriever","Labrador Retriever Mix","Newfoundland"]
            },
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,25))
        dff = df
        titleLabels =['All','Water Rescue','Mountain or Wilderness Rescue','Disaster Rescue or Individual Tracking']
        titleVal = titleLabels[value]
        barchart=px.bar(
            data_frame=dff,        
            x='name',
            y='age_upon_outcome_in_weeks', 
            labels={
                'name':'Names',
                'age_upon_outcome_in_weeks':'Age',
                'breed':'Breed',
                'outcome_type':'Status'
            },
            hover_data = {
                'age_upon_outcome_in_weeks',
                'name',
                'breed',
                'outcome_type'
            },
            title=titleVal
    )
    elif int(value)==2:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]
            },
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,25))
        dff = df
        titleLabels =['All','Water Rescue','Mountain or Wilderness Rescue','Disaster Rescue or Individual Tracking']
        titleVal = titleLabels[value]
        barchart=px.bar(
            data_frame=dff,        
            x='name',
            y='age_upon_outcome_in_weeks', 
            labels={
                'name':'Names',
                'age_upon_outcome_in_weeks':'Age',
                'breed':'Breed',
                'outcome_type':'Status'
            },
            hover_data = {
                'age_upon_outcome_in_weeks',
                'name',
                'breed',
                'outcome_type'
            },
            title=titleVal
    )
    elif int(value)==3:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]
            },
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20,"$lte":300}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,25))
        dff = df
        titleLabels =['All','Water Rescue','Mountain or Wilderness Rescue','Disaster Rescue or Individual Tracking']
        titleVal = titleLabels[value]
        barchart=px.bar(
            data_frame=dff,        
            x='name',
            y='age_upon_outcome_in_weeks', 
            labels={
                'name':'Names',
                'age_upon_outcome_in_weeks':'Age',
                'breed':'Breed',
                'outcome_type':'Status'
            },
            hover_data = {
                'age_upon_outcome_in_weeks',
                'name',
                'breed',
                'outcome_type'
            },
            title=titleVal
    )
    return (barchart)
    

## UPDATE TABLE INTERACTIONS    

## COLUMN SELECTION LOGIC
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#FILTERS
@app.callback(
    Output('datatable-id','data'),
    [
        Input('rescue-filters','value'),
    ]
)

def selected_filter(value):
    if int(value)==0:
        data = {}
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
    elif int(value)==1:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["Chesapeake Bay Retriever","Labrador Retriever Mix","Newfoundland"]
            },
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
    elif int(value)==2:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]
            },
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
    elif int(value)==3:
        data = {
            "animal_type":"Dog",
            "breed":{
                "$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]
            },
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20,"$lte":300}
        }
        df = pd.DataFrame.from_records(shelter.returnAll(data,0))
    return df.to_dict('records')

    
app